# Prefix Tuning 과정 실습
____

* 파이토치 / 트랜스포머 라이브러리 로드

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [45]:
# 모델과 토크나이저 로드
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [46]:
# 입력 문장과 라벨 정의
input_texts = ["The movie was great!", "The movie was terrible.", "I loved the acting.", "The plot was boring."]
labels = [1, 0, 1, 0]  # 예시로 긍정(1), 부정(0) 라벨 사용

# 입력 텍스트를 토큰화하고 텐서로 변환
inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True, max_length=10)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
labels = torch.tensor(labels)

* 기본 BERT 모델 학습

In [47]:
basic_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(basic_model.parameters(), lr=1e-5)

# 학습 루프
num_epochs = 3
for epoch in range(num_epochs):
    basic_model.train()  # 모델을 학습 모드로 설정
    
    optimizer.zero_grad()  # 옵티마이저의 기울기 초기화
    
    # 모델의 출력 계산
    outputs = basic_model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    
    # 손실 계산
    loss = criterion(logits, labels)
    
    # 역전파를 통해 기울기 계산
    loss.backward()
    
    # 옵티마이저를 통해 파라미터 업데이트
    optimizer.step()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.6899866461753845
Epoch 2/3, Loss: 0.6152526140213013
Epoch 3/3, Loss: 0.5844820737838745


* Prefix-튜닝 모델링

In [34]:
class PrefixTuningModel(nn.Module):
    def __init__(self, model_name, prefix_length):
        super().__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.prefix_length = prefix_length  # prefix_length를 클래스 속성으로 저장
        self.prefix_embeddings = nn.Parameter(torch.randn((self.model.config.num_hidden_layers, prefix_length, self.model.config.hidden_size)))

    def forward(self, input_ids, attention_mask):
        batch_size = input_ids.size(0)
        # (num_hidden_layers, 1, prefix_length, hidden_size) 형태로 변환
        # (num_hidden_layers, batch_size, prefix_length, hidden_size)로 확장
        prefix_embeds = self.prefix_embeddings.unsqueeze(1).expand(-1, batch_size, -1, -1)
        
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_states = list(outputs.hidden_states)

        ########## 각 은닉 상태에 프리픽스 임베딩 결합 #############
        for i in range(1, len(hidden_states)):  # hidden_states[0]은 입력 임베딩임
            hidden_states[i] = torch.cat((prefix_embeds[i-1], hidden_states[i]), dim=1)
            
        # 확장된 attention_mask 생성
        extended_attention_mask = torch.cat([
            torch.ones((batch_size, self.prefix_length), dtype=attention_mask.dtype, device=attention_mask.device),
            attention_mask
        ], dim=1)
        
        # 결합된 은닉 상태를 사용하여 최종 출력 계산
        inputs_embeds = hidden_states[-1]
        return self.model(inputs_embeds=inputs_embeds, attention_mask=extended_attention_mask)

* 튜닝

In [48]:
# 손실 함수와 옵티마이저 정의
prefix_model = PrefixTuningModel(model_name, prefix_length=5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(prefix_model.parameters(), lr=1e-5)

# 학습 루프
for epoch in range(num_epochs):
    prefix_model.train()  # 모델을 학습 모드로 설정
    
    optimizer.zero_grad()  # 옵티마이저의 기울기 초기화
    
    # 모델의 출력 계산
    outputs = prefix_model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    
    # 손실 계산
    loss = criterion(logits, labels)
    
    # 역전파를 통해 기울기 계산
    loss.backward()
    
    # 옵티마이저를 통해 파라미터 업데이트
    optimizer.step()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.7236154675483704
Epoch 2/3, Loss: 0.6990016102790833
Epoch 3/3, Loss: 0.7475633025169373


In [39]:
inputs

{'input_ids': tensor([[ 101, 1996, 3185, 2001, 2307,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

* 비교 평가

In [49]:
# 평가 함수 정의
def evaluate_model(model, input_ids, attention_mask, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        accuracy = (predicted_labels == labels).float().mean().item()
        return accuracy

# 평가 데이터 준비
eval_input_texts = ["The movie was fantastic!", "The plot was dull."]
eval_labels = torch.tensor([1, 0])
eval_inputs = tokenizer(eval_input_texts, return_tensors='pt', padding=True, truncation=True, max_length=10)
eval_input_ids = eval_inputs['input_ids']
eval_attention_mask = eval_inputs['attention_mask']

# 기본 모델 평가
basic_model_accuracy = evaluate_model(basic_model, eval_input_ids, eval_attention_mask, eval_labels)
print(f"Basic Model Accuracy: {basic_model_accuracy}")

# Prefix Tuning 모델 평가
prefix_model_accuracy = evaluate_model(prefix_model, eval_input_ids, eval_attention_mask, eval_labels)
print(f"Prefix Tuning Model Accuracy: {prefix_model_accuracy}")

Basic Model Accuracy: 1.0
Prefix Tuning Model Accuracy: 1.0


## NLP에서 추가로 알아둬야 할 중간변수들
___

### 은닉층 및 텐서 차원 관련
___

In [ ]:
print('num_hidden_layers:', model.model.config.num_hidden_layers)
# 트랜스포머 모델의 인코더(또는 디코더) 계층의 수
# 트랜스포머 모델은 여러 개의 인코더 및 디코더 계층으로 구성
# 각 계층은 셀프 어텐션과 피드포워드 신경망으로 구성됨
# 계층이 많을수록 모델은 더 복잡한 패턴을 학습할수 있음.
# 여기서는 12개의 인코더 계층이 있다는 것을 의미
print('hidden_size:', model.model.config.hidden_size)
# 각 계층의 은닉 상태 벡터의 차원. BERT-base 모델에서 hidden_size=768은 각 토큰이 768차원 벡터로 표현된다는 것을 의미
print('prefix_length:', model.prefix_embeddings.size(1))
# prefix_length는 5로 설정했으므로, 5개의 토큰으로 이루어진 prefix를 사용

num_hidden_layers: 12
hidden_size: 768
prefix_length: 5


In [ ]:
# 3-D tensor를 생성 ( 12 x 5 x 768 )
model.prefix_embeddings.shape

torch.Size([12, 5, 768])

### input_ids
___
- 입력 텍스트를 토큰화한 후, 각 토큰을 고유한 정수 ID로 변환한 것. 
- 토크나이저는 텍스트를 사전에 정의된 어휘(vocabulary)에 따라 토큰으로 분할하고, 각 토큰을 어휘에서의 인덱스로 매핑

예를 들어, BERT 모델의 토크나이저를 사용하면 다음과 같은 과정이 진행됨.

1. 텍스트: "The movie was great!"
2. 토큰화: `["[CLS]", "the", "movie", "was", "great", "!", "[SEP]"]`
3. 정수 ID 변환: `[101, 1996, 3185, 2001, 2307, 999, 102]`

 `101`은 `[CLS]` 토큰의 ID이고, `102`는 `[SEP]` 토큰의 ID
`[CLS]` 토큰은 문장의 시작을 나타내며, `[SEP]` 토큰은 문장의 끝을 나타냄.

### attention_mask
___
- 입력 시퀀스에서 실제 단어와 패딩 토큰을 구분하는 데 사용됨.
- 패딩 토큰은 입력 시퀀스 길이를 일정하게 맞추기 위해 추가된 토큰. 
- `attention_mask`는 각 토큰 위치가 실제 단어인지 패딩인지 나타내는 바이너리 값의 시퀀스임.

예를 들어, 입력 시퀀스의 최대 길이가 10이고, 실제 입력이 더 짧은 경우 패딩을 추가.

1. 실제 입력: `[101, 1996, 3185, 2001, 2307, 999, 102]`
2. 패딩 추가 후: `[101, 1996, 3185, 2001, 2307, 999, 102, 0, 0, 0]`
3. `attention_mask`: `[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]`

여기서 `1`은 실제 단어 위치를 나타내고, `0`은 패딩 위치를 나타냄.